In [2]:
import pandas as pd
import plotly.express as px
import warnings 
from internproject.constants import ROOT_DATA_DIR

add_lp = pd.read_parquet(ROOT_DATA_DIR / "stETH_ETH_add_lp.parquet")  # more details on the add liquidity events
remove_lp = pd.read_parquet(
    ROOT_DATA_DIR / "stETH_ETH_remove_lp.parquet"
)  # more details on the remove liquidity events
lp_transfers = pd.read_parquet(ROOT_DATA_DIR / "stETH_ETH_lp_transfers.parquet")
zapper_address = '0x271fbE8aB7f1fB262f81C77Ea5303F03DA9d3d6A'


In [3]:
stETH_ETH_apr_df = pd.read_csv(
    ROOT_DATA_DIR
    / 'daily_snapshots/0xDC24316b9AE028F1497c275EB9192a3Ea0f67022_apr_df.csv',
    index_col=0,
    parse_dates=True,
).dropna()

# finding rolled avg apr from og steth eth dataframe and set it as price 
stETH_ETH_apr_df['cur_naive_apr'] = stETH_ETH_apr_df[['raw_base_apr', 'raw_fee_apr', 'raw_incentive_apr', 'raw_price_apr']].sum(axis=1)
stETH_ETH_apr_df['rolling_cur_naive_apr'] = stETH_ETH_apr_df['cur_naive_apr'].rolling(7).mean()
stETH_ETH_apr_df['rolling_cur_naive_apr'].median()

5.086284024877622

### Reward rate data 

In [4]:
from internproject.constants import ROOT_DATA_DIR
stETH_ETH_incentive_rewards_df = pd.read_parquet(ROOT_DATA_DIR / 'raw_pool_data/curve_stETH_ETH/extra_rewards_df.parquet')
px.line(stETH_ETH_incentive_rewards_df['reward_rate'])
# this is the quanitty of LDO rewards issued per second.
# This can be used a replacement for APR that is not circular

In [5]:
px.line(stETH_ETH_incentive_rewards_df['reward_rate_eth'])

In [6]:
null_address = "0x0000000000000000000000000000000000000000"

### Understanding lp transfers

1. The "value" column in `lp_transfers` refers to the quantity of lp tokens
2. If the null address is in the "from" column, then it is adding liquidity to the pool. 
3. If the null address is in the "to" column, then it is removeing liquidity from the pool.
4. It is safe to ignore the other transfers

In [7]:
add_liquidity_transfers = lp_transfers[lp_transfers["from"] == null_address]
withdraw_liquidity_transfers = lp_transfers[lp_transfers["to"] == null_address]

In [8]:
withdraw_liquidity_transfers

,from,to,value,event,block,transaction_index,log_index,hash,timestamp
timestamp,,,,,,,,,
2021-01-05 17:12:33,0x3a63aA8AE11Ecaa82fC352d0390634E8030B2ffd,0x0000000000000000000000000000000000000000,2.000945,Transfer,11595786,133,295,0xbd58a23237562e0332aaa7a7668f8dd71e4aecddc95d...,2021-01-05 17:12:33
2021-01-05 17:46:38,0xaF297deC752c909092A117A932A8cA4AaaFF9795,0x0000000000000000000000000000000000000000,499.501433,Transfer,11595955,11,5,0xad790ed2cb537281ae0c7eaf1558b1494b869e7f7e1c...,2021-01-05 17:46:38
2021-01-05 21:03:54,0xb084F0AdB86BF6B001E540Ac05439D98A499ce1D,0x0000000000000000000000000000000000000000,0.999754,Transfer,11596894,75,96,0xc485ceb5703ddaf5393049b8fb71a06b20ee0c0b4ced...,2021-01-05 21:03:54
2021-01-06 03:36:23,0x89B8B20AE90328692cD367f75aaFadF55fd33E8B,0x0000000000000000000000000000000000000000,10.037649,Transfer,11598604,77,130,0xbebdcd6fa25430bea017919704fd3dcd12f55ef0f99b...,2021-01-06 03:36:23
2021-01-06 12:17:18,0xdE3E412d4fe3c9d90ac74d0A9B064951B39EEae4,0x0000000000000000000000000000000000000000,1.002055,Transfer,11600957,56,66,0x5a33f4f9e2e2a55f09141909f75b1b308d092183681c...,2021-01-06 12:17:18
...,...,...,...,...,...,...,...,...,...
2023-06-20 22:25:35,0xEa508F82728927454bd3ce853171b0e2705880D4,0x0000000000000000000000000000000000000000,0.000000,Transfer,17523858,45,95,0x6188d0177bb9a8a4a3bb5feb560477d6ec4642c9fde0...,2023-06-20 22:25:35
2023-06-20 22:33:11,0x46D5F548E02fdee3ea7E4168B7cF896429fE6fe7,0x0000000000000000000000000000000000000000,65.033669,Transfer,17523896,78,240,0x113260903e90d948871a8a0af2f03156c17781a455ea...,2023-06-20 22:33:11
2023-06-20 22:40:35,0x5aD15EBa9D4E9351414084Eb1dd1EACc2068A4A7,0x0000000000000000000000000000000000000000,51.167100,Transfer,17523933,42,101,0x39af71a3003ae573c6739b9679bede548ea7cc53a6b0...,2023-06-20 22:40:35


### Analyzing behavior and liquidity of unique users 
- combining the added and withdrawed liquidty 

In [9]:
withdraw_liquidity_transfers['value'] = withdraw_liquidity_transfers['value']* -1

warnings.filterwarnings('ignore')
withdraw_liquidity_transfers['value']

/var/folders/d9/vxfbfh9572v4lvm_hy19jbsh0000gn/T/ipykernel_22450/3396211988.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



timestamp
2021-01-05 17:12:33     -2.000945
2021-01-05 17:46:38   -499.501433
2021-01-05 21:03:54     -0.999754
2021-01-06 03:36:23    -10.037649
2021-01-06 12:17:18     -1.002055
                          ...    
2023-06-20 22:25:35     -0.000000
2023-06-20 22:33:11    -65.033669
2023-06-20 22:40:35    -51.167100
2023-06-20 23:28:11     -0.000000
2023-06-21 00:00:11     -0.003230
Name: value, Length: 27851, dtype: float64

In [10]:
add_liquidity_transfers['user'] = add_liquidity_transfers['to']
withdraw_liquidity_transfers['user'] = withdraw_liquidity_transfers['from']

transfer_df = pd.concat([add_liquidity_transfers, withdraw_liquidity_transfers])
transfer_df.columns

warnings.filterwarnings('ignore')

finding users who first added and first removed 

In [11]:
users_first_add = transfer_df[transfer_df['value'] > 0].groupby('user')[['timestamp']].min()
users_first_add.columns = ['date_added']
users_first_remove = transfer_df[transfer_df['value'] < 0].groupby('user')[['timestamp']].min()
users_first_remove.columns = ['date_removed']
users_df = users_first_add.join(users_first_remove, how='outer')
users_df['date_removed'] = users_df['date_removed'].fillna(pd.to_datetime('2050-01-01'))
users_df

,date_added,date_removed
user,,
0x0000000000117D8C0c6103ecA077462224723192,NaT,2022-09-17 07:09:47
0x000000000037D42ab4e2226CE6f44C5dC0Cf5b16,NaT,2022-05-15 00:02:54
0x00000000245A821a3e646b8187636AD09Cd9cCB0,NaT,2022-10-16 19:37:59
0x000000003fBC5761A6849dC46d6C292c7B01e894,NaT,2023-06-01 10:00:59
0x0000000090E2bA937F0b5615b57409a8A6a1D6Dd,2021-08-15 14:08:51,2021-08-15 14:47:39
...,...,...
0xff879CA9C84D976dB7E0180436e3a689eD70E1D2,2022-11-27 02:36:23,2050-01-01 00:00:00
0xff8a51f0dE0166672a2f77f0213e6D814c28E36C,2021-06-29 23:09:22,2021-06-29 22:51:41
0xff9A717FCF03318485579c4b553820eDf73aDf0A,2021-09-15 02:57:16,2021-11-05 08:17:51


In [12]:
users_who_never_added = users_df[users_df['date_added'].isna()].index
users_who_removed_before_they_added = users_df[users_df['date_removed'] < users_df['date_added']].index
print("Users who never added liquidity: {}\nUsers who removed liquidity before they added: {}"
      .format((len(users_who_never_added)), len(users_who_removed_before_they_added)))

Users who never added liquidity: 1298
Users who removed liquidity before they added: 114


In [13]:
num_total_unique_users = transfer_df['user'].nunique()
num_total_unique_users
print("Total number of unique users: {}".format(num_total_unique_users))

Total number of unique users: 16152


In [14]:
transfer_df[transfer_df['user'] == users_who_never_added[0]][['from', 'to', 'value']]
# this user never added liquidity and only withdrew. 

,from,to,value
timestamp,,,
2022-09-17 07:09:47,0x0000000000117D8C0c6103ecA077462224723192,0x0000000000000000000000000000000000000000,-0.308512


In [15]:
transfer_df[transfer_df['user'] == users_who_removed_before_they_added[0]][['from', 'to', 'value']]
# this person withdrew 20 eth before adding, then added 20 later on

,from,to,value
timestamp,,,
2022-11-06 05:49:35,0x0000000000000000000000000000000000000000,0x01e6fd0aE73D9194b19f9B376065577927A0D5f5,20.789995
2022-06-20 13:20:23,0x01e6fd0aE73D9194b19f9B376065577927A0D5f5,0x0000000000000000000000000000000000000000,-20.314335


In [16]:
# locating one unique user
transfer_df['user'].value_counts().head(100).tail(1)

0xb76edb86C89f241aE87CE7fE9e48E10588CA1D0D    25
Name: user, dtype: int64

In [17]:
transfer_df['user'] == '0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7'

timestamp
2021-01-05 00:26:18    False
2021-01-05 11:26:21    False
2021-01-05 13:35:18     True
2021-01-05 13:48:47     True
2021-01-05 14:32:47    False
                       ...  
2023-06-20 22:25:35    False
2023-06-20 22:33:11    False
2023-06-20 22:40:35    False
2023-06-20 23:28:11    False
2023-06-21 00:00:11    False
Name: user, Length: 62068, dtype: bool

In [18]:
transfer_df_1person = transfer_df[transfer_df['user'] == '0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7']

In [19]:
transfer_df_1person

,from,to,value,event,block,transaction_index,log_index,hash,timestamp,user
timestamp,,,,,,,,,,
2021-01-05 13:35:18,0x0000000000000000000000000000000000000000,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,69.999492,Transfer,11594829,94,97,0x8eca09399384c600131f1f9e1d931246e9129f2e1e11...,2021-01-05 13:35:18,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7
2021-01-05 13:48:47,0x0000000000000000000000000000000000000000,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,1999.989998,Transfer,11594889,122,123,0x6c6cb813a1cfee7cb6bb241b74a961ede8b70eed5ae3...,2021-01-05 13:48:47,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7
2021-01-13 13:27:28,0x0000000000000000000000000000000000000000,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,1790.364658,Transfer,11646866,51,89,0xce8992a8ad1e9f650ed78f8d387f507ce0d420719776...,2021-01-13 13:27:28,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7
2021-04-10 19:41:29,0x0000000000000000000000000000000000000000,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,3718.426815,Transfer,12214131,160,323,0xa2d93409e7514d01c04231d17b5248ab5e0947dcdfff...,2021-04-10 19:41:29,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7
2021-02-01 22:02:17,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,0x0000000000000000000000000000000000000000,-199.860373,Transfer,11772899,44,63,0xef7c06a31d0fb37afe80300cbbb25b0b51f71a8ab162...,2021-02-01 22:02:17,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7
2021-03-08 14:49:11,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,0x0000000000000000000000000000000000000000,-3660.493775,Transfer,11998460,184,170,0xd21e15bceedcc71042088f39a39ab4bffd5d3f750351...,2021-03-08 14:49:11,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7
2022-12-24 16:04:23,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,0x0000000000000000000000000000000000000000,-3718.426815,Transfer,16255668,11,62,0xd6bef2e8d1fef6f333730b95e9b1c0de5da4a656c242...,2022-12-24 16:04:23,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7


In [20]:
# finding cumulative sum of this person's transfer payments, which is the same as their balance over time
transfer_df_1person['value'].cumsum().round(3).values
transfer_df_1person['balance_over_time'] = transfer_df_1person['value'].cumsum().round(3).values

# merge reward rate df with this df so we can access the reward rates for this whale's transfers
transfer_df_1person_reward = pd.merge(transfer_df_1person, stETH_ETH_incentive_rewards_df, left_on=transfer_df_1person.index, right_on=stETH_ETH_incentive_rewards_df.index, how='left')

warnings.filterwarnings('ignore')

In [21]:
transfer_df_1person

,from,to,value,event,block,transaction_index,log_index,hash,timestamp,user,balance_over_time
timestamp,,,,,,,,,,,
2021-01-05 13:35:18,0x0000000000000000000000000000000000000000,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,69.999492,Transfer,11594829,94,97,0x8eca09399384c600131f1f9e1d931246e9129f2e1e11...,2021-01-05 13:35:18,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,69.999
2021-01-05 13:48:47,0x0000000000000000000000000000000000000000,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,1999.989998,Transfer,11594889,122,123,0x6c6cb813a1cfee7cb6bb241b74a961ede8b70eed5ae3...,2021-01-05 13:48:47,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,2069.989
2021-01-13 13:27:28,0x0000000000000000000000000000000000000000,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,1790.364658,Transfer,11646866,51,89,0xce8992a8ad1e9f650ed78f8d387f507ce0d420719776...,2021-01-13 13:27:28,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,3860.354
2021-04-10 19:41:29,0x0000000000000000000000000000000000000000,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,3718.426815,Transfer,12214131,160,323,0xa2d93409e7514d01c04231d17b5248ab5e0947dcdfff...,2021-04-10 19:41:29,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,7578.781
2021-02-01 22:02:17,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,0x0000000000000000000000000000000000000000,-199.860373,Transfer,11772899,44,63,0xef7c06a31d0fb37afe80300cbbb25b0b51f71a8ab162...,2021-02-01 22:02:17,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,7378.921
2021-03-08 14:49:11,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,0x0000000000000000000000000000000000000000,-3660.493775,Transfer,11998460,184,170,0xd21e15bceedcc71042088f39a39ab4bffd5d3f750351...,2021-03-08 14:49:11,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,3718.427
2022-12-24 16:04:23,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,0x0000000000000000000000000000000000000000,-3718.426815,Transfer,16255668,11,62,0xd6bef2e8d1fef6f333730b95e9b1c0de5da4a656c242...,2022-12-24 16:04:23,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,-0.000


In [22]:
transfer_df_1person_reward

,key_0,from,to,value,event,block_x,transaction_index,log_index,hash,timestamp,user,balance_over_time,lp_token_total_staked,reward_rate,block_y,reward_rate_eth
0,2021-01-05 13:35:18,0x0000000000000000000000000000000000000000,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,69.999492,Transfer,11594829,94,97,0x8eca09399384c600131f1f9e1d931246e9129f2e1e11...,2021-01-05 13:35:18,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,69.999,NaN,NaN,NaN,NaN
1,2021-01-05 13:48:47,0x0000000000000000000000000000000000000000,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,1999.989998,Transfer,11594889,122,123,0x6c6cb813a1cfee7cb6bb241b74a961ede8b70eed5ae3...,2021-01-05 13:48:47,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,2069.989,NaN,NaN,NaN,NaN
2,2021-01-13 13:27:28,0x0000000000000000000000000000000000000000,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,1790.364658,Transfer,11646866,51,89,0xce8992a8ad1e9f650ed78f8d387f507ce0d420719776...,2021-01-13 13:27:28,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,3860.354,NaN,NaN,NaN,NaN
3,2021-04-10 19:41:29,0x0000000000000000000000000000000000000000,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,3718.426815,Transfer,12214131,160,323,0xa2d93409e7514d01c04231d17b5248ab5e0947dcdfff...,2021-04-10 19:41:29,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,7578.781,NaN,NaN,NaN,NaN
4,2021-02-01 22:02:17,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,0x0000000000000000000000000000000000000000,-199.860373,Transfer,11772899,44,63,0xef7c06a31d0fb37afe80300cbbb25b0b51f71a8ab162...,2021-02-01 22:02:17,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,7378.921,NaN,NaN,NaN,NaN
5,2021-03-08 14:49:11,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,0x0000000000000000000000000000000000000000,-3660.493775,Transfer,11998460,184,170,0xd21e15bceedcc71042088f39a39ab4bffd5d3f750351...,2021-03-08 14:49:11,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,3718.427,NaN,NaN,NaN,NaN
6,2022-12-24 16:04:23,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,0x0000000000000000000000000000000000000000,-3718.426815,Transfer,16255668,11,62,0xd6bef2e8d1fef6f333730b95e9b1c0de5da4a656c242...,2022-12-24 16:04:23,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,-0.000,NaN,NaN,NaN,NaN


In [23]:
# amount of tokens supplied (balance) over time 
px.bar(transfer_df_1person_reward['balance_over_time'])

### WHALES!!

In [24]:
# grouping df by user and calculating each individual's cumulative sum to FIND WHALES 
transfer_df.sort_index(inplace=True)
transfer_df['cumsol_col'] = transfer_df.groupby(['user'])['value'].cumsum()

# create pivot table that contains unique users and their cumulative sum 
#pivot_df = transfer_df.pivot_table(values='cumsol_col', index=transfer_df.index, columns='user')
# pivot_df

In [25]:
parsed_rows = transfer_df[transfer_df['user'].isin(users_first_add.index)]
parsed_rows
# transfer_df = transfer_df - parsed_rows

,from,to,value,event,block,transaction_index,log_index,hash,timestamp,user,cumsol_col
timestamp,,,,,,,,,,,
2021-01-05 11:26:21,0x0000000000000000000000000000000000000000,0x6ab615CF8deCFc488186E54066Fc10589C9293A3,0.160000,Transfer,11594223,141,250,0x16d97e10e582735b4731aaec2823f781edb3355fa9b2...,2021-01-05 11:26:21,0x6ab615CF8deCFc488186E54066Fc10589C9293A3,0.160000
2021-01-05 13:35:18,0x0000000000000000000000000000000000000000,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,69.999492,Transfer,11594829,94,97,0x8eca09399384c600131f1f9e1d931246e9129f2e1e11...,2021-01-05 13:35:18,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,69.999492
2021-01-05 13:48:47,0x0000000000000000000000000000000000000000,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,1999.989998,Transfer,11594889,122,123,0x6c6cb813a1cfee7cb6bb241b74a961ede8b70eed5ae3...,2021-01-05 13:48:47,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,2069.989490
2021-01-05 14:32:47,0x0000000000000000000000000000000000000000,0xb084F0AdB86BF6B001E540Ac05439D98A499ce1D,0.999754,Transfer,11595091,101,132,0xa3ffd6f2c890037b38c11d64e34e991fa1f31318e873...,2021-01-05 14:32:47,0xb084F0AdB86BF6B001E540Ac05439D98A499ce1D,0.999754
2021-01-05 14:37:20,0x0000000000000000000000000000000000000000,0x90A7D7a5abfa5096bCef8dE95dD44138704Ab3C8,1.199793,Transfer,11595113,122,145,0x9b10763d9922e9d583fe90e02e520cba9d0ae4ecabd5...,2021-01-05 14:37:20,0x90A7D7a5abfa5096bCef8dE95dD44138704Ab3C8,1.199793
...,...,...,...,...,...,...,...,...,...,...,...
2023-06-20 20:37:35,0x3aE7486eE0832bebD54b58BaBbD2A23EC713195F,0x0000000000000000000000000000000000000000,-5.228781,Transfer,17523323,161,325,0xc3cac1558a81e67007885432be0fb20e71859227b0e2...,2023-06-20 20:37:35,0x3aE7486eE0832bebD54b58BaBbD2A23EC713195F,-0.326656
2023-06-20 22:31:59,0x0000000000000000000000000000000000000000,0x5aD15EBa9D4E9351414084Eb1dd1EACc2068A4A7,51.167100,Transfer,17523890,48,92,0xd832e4bcce6354ead8e76a140b3ba3066cf81b2955da...,2023-06-20 22:31:59,0x5aD15EBa9D4E9351414084Eb1dd1EACc2068A4A7,51.167100
2023-06-20 22:33:11,0x46D5F548E02fdee3ea7E4168B7cF896429fE6fe7,0x0000000000000000000000000000000000000000,-65.033669,Transfer,17523896,78,240,0x113260903e90d948871a8a0af2f03156c17781a455ea...,2023-06-20 22:33:11,0x46D5F548E02fdee3ea7E4168B7cF896429fE6fe7,0.000000


In [26]:
# locate whale by finding where at one point user had over a certain amount
whale_df = transfer_df.loc[transfer_df['cumsol_col'] > 50000]

# find how many unique whales there are 
num_unique_whales = whale_df.groupby(['user']) 
len(num_unique_whales.groups)
# there are 504 unique whales if we define them as those who have cumulative balance over 1000

21

##### cumsol sorted smallest to largest

In [27]:
transfer_df.groupby('user').max().sort_values('cumsol_col')

,from,to,value,event,block,transaction_index,log_index,hash,timestamp,cumsol_col
user,,,,,,,,,,
0xf44Ac73b957B28207504Ee2fd5d51eCbfeF7d8fF,0xf44Ac73b957B28207504Ee2fd5d51eCbfeF7d8fF,0x0000000000000000000000000000000000000000,-80752.602372,Transfer,16038263,0,0,0x2166dcfa7f9a828635963de909c78fd3bee673bcbf13...,2022-11-24 07:11:23,-80752.602372
0x5AC6A7CAc78197ef80F64D076b8Bfee2574DCd5b,0x5AC6A7CAc78197ef80F64D076b8Bfee2574DCd5b,0x0000000000000000000000000000000000000000,-19901.074656,Transfer,15966529,14,34,0xe56afad8c1557b19d87989037131406484c9577bde57...,2022-11-14 06:37:23,-19901.074656
0xA0Da148b6CcAde41ab551f8cED537A6c436BAaFa,0xA0Da148b6CcAde41ab551f8cED537A6c436BAaFa,0x0000000000000000000000000000000000000000,-13000.000000,Transfer,14615872,80,129,0xed4804f8260f468f8bf7dbe24c081e57fa9fe1f117f8...,2022-04-19 13:52:55,-13000.000000
0x6a522F7ad8Ffe69e2B076e13c73dc698eFbd8044,0x6a522F7ad8Ffe69e2B076e13c73dc698eFbd8044,0x0000000000000000000000000000000000000000,-6457.421531,Transfer,14179288,42,73,0x321427338678b6b2eb4c4a3751cab14e15c763fec7b4...,2022-02-10 16:04:10,-6457.421531
0x6e1D0fEb04281d1a96DFE5D139941bD86c94712f,0x6e1D0fEb04281d1a96DFE5D139941bD86c94712f,0x0000000000000000000000000000000000000000,-4604.795480,Transfer,17379760,73,131,0x44f3fdac141b11e692489ba91ecd070e3cc45139b0aa...,2023-05-31 15:31:47,-4604.795480
...,...,...,...,...,...,...,...,...,...,...
0x12b5c9191E186658841F2431943c47278f68E075,0x12b5c9191E186658841F2431943c47278f68E075,0x12b5c9191E186658841F2431943c47278f68E075,86061.625846,Transfer,14937471,273,587,0xef05f74c37b9b32cbfdaf80933c076b344a00af86bb0...,2022-06-10 08:37:37,105327.329795
0x9Cf59Ecde926cc1B5FF1Cb9b247E0FEC627091b1,0x9Cf59Ecde926cc1B5FF1Cb9b247E0FEC627091b1,0x9Cf59Ecde926cc1B5FF1Cb9b247E0FEC627091b1,60862.175792,Transfer,13188689,269,356,0xff8570fa3c8b4b43055ebc7bd984ab412e68a8295530...,2021-09-09 01:44:23,114142.222772
0xEAA7723633cf598E872D611f5EC50a45b65CBc72,0xEAA7723633cf598E872D611f5EC50a45b65CBc72,0xEAA7723633cf598E872D611f5EC50a45b65CBc72,114348.306873,Transfer,17512484,382,1100,0xf2c693113c6e9693eced243131d5e7dfe43c3148cf74...,2023-06-19 08:04:23,118845.280202


In [28]:
whales_reward_time = transfer_df.groupby('user').max().sort_values('cumsol_col')

In [29]:
whale_df

,from,to,value,event,block,transaction_index,log_index,hash,timestamp,user,cumsol_col
timestamp,,,,,,,,,,,
2021-04-10 17:44:19,0x0000000000000000000000000000000000000000,0xdb5AA12AD695Ef2a28C6CdB69f2BB04BEd20a48e,31160.119571,Transfer,12213597,65,82,0xbc2b8f4aa968697352f7ab82f6db0dc30f0eb3b0559d...,2021-04-10 17:44:19,0xdb5AA12AD695Ef2a28C6CdB69f2BB04BEd20a48e,56051.962624
2021-04-12 19:07:06,0x0000000000000000000000000000000000000000,0xdb5AA12AD695Ef2a28C6CdB69f2BB04BEd20a48e,34577.303605,Transfer,12227029,0,0,0x277a7d200414999879bd2934b06b7cd144b62d181793...,2021-04-12 19:07:06,0xdb5AA12AD695Ef2a28C6CdB69f2BB04BEd20a48e,90629.266229
2021-04-13 17:33:41,0x0000000000000000000000000000000000000000,0xdb5AA12AD695Ef2a28C6CdB69f2BB04BEd20a48e,12772.157944,Transfer,12233123,161,183,0x0236cf92c4cca2673faea6a29d538c75cbbc9447ca05...,2021-04-13 17:33:41,0xdb5AA12AD695Ef2a28C6CdB69f2BB04BEd20a48e,103401.424173
2021-04-21 22:35:34,0x0000000000000000000000000000000000000000,0xdb5AA12AD695Ef2a28C6CdB69f2BB04BEd20a48e,13208.519255,Transfer,12286307,6,9,0xf1473c518b2a16f07411ab22a760bc3e88d0bbde1f50...,2021-04-21 22:35:34,0xdb5AA12AD695Ef2a28C6CdB69f2BB04BEd20a48e,116609.943428
2021-05-07 13:08:27,0x0000000000000000000000000000000000000000,0xdb5AA12AD695Ef2a28C6CdB69f2BB04BEd20a48e,29569.346367,Transfer,12387319,0,0,0xa7f2545100be96c67e71124ce06eda1d9eb5e2aac3c4...,2021-05-07 13:08:27,0xdb5AA12AD695Ef2a28C6CdB69f2BB04BEd20a48e,146179.289795
...,...,...,...,...,...,...,...,...,...,...,...
2023-06-16 13:05:35,0x0000000000000000000000000000000000000000,0x271fbE8aB7f1fB262f81C77Ea5303F03DA9d3d6A,46.525736,Transfer,17492581,124,166,0x248bb5fe3adc7f6ca6c60541fcd88073c25dcee2dd28...,2023-06-16 13:05:35,0x271fbE8aB7f1fB262f81C77Ea5303F03DA9d3d6A,57576.061964
2023-06-17 21:10:47,0x0000000000000000000000000000000000000000,0x271fbE8aB7f1fB262f81C77Ea5303F03DA9d3d6A,3.721728,Transfer,17502120,63,152,0xe05cde3d759ade71ff1948c18eeea9398148a3da3a39...,2023-06-17 21:10:47,0x271fbE8aB7f1fB262f81C77Ea5303F03DA9d3d6A,57579.783692
2023-06-17 21:18:11,0x0000000000000000000000000000000000000000,0x271fbE8aB7f1fB262f81C77Ea5303F03DA9d3d6A,18.608794,Transfer,17502157,87,226,0x015d8d951dfd2a2928e80c62b3ce62afaed808f3bd98...,2023-06-17 21:18:11,0x271fbE8aB7f1fB262f81C77Ea5303F03DA9d3d6A,57598.392485


#### Behavior of largest whale in the pool (for which we have reward rate data for)

In [30]:
largest_whale = transfer_df[transfer_df['user'] == '0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3']
largest_whale

,from,to,value,event,block,transaction_index,log_index,hash,timestamp,user,cumsol_col
timestamp,,,,,,,,,,,
2022-09-26 14:36:23,0x0000000000000000000000000000000000000000,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,118782.299389,Transfer,15618180,367,385,0x200853bc39a0b4ac57f3a396d22a39a070686ec5e63a...,2022-09-26 14:36:23,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,118782.299389
2022-09-26 19:08:35,0x0000000000000000000000000000000000000000,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,0.921205,Transfer,15619528,39,152,0xe62463f53c97cc3542eb72887719022b33172209954b...,2022-09-26 19:08:35,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,118783.220593
2022-09-27 04:24:35,0x0000000000000000000000000000000000000000,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,1.909552,Transfer,15622291,162,246,0xfb072e072c6557451effe6a61158eed4360decd747c2...,2022-09-27 04:24:35,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,118785.130146
2022-09-27 13:44:23,0x0000000000000000000000000000000000000000,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,1.903760,Transfer,15625068,172,316,0x21ba8a9e96c6d327fb6df54ff59a683ed8dc418fa2c7...,2022-09-27 13:44:23,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,118787.033906
2022-09-27 23:17:23,0x0000000000000000000000000000000000000000,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,1.905536,Transfer,15627904,76,160,0xa1add66f1689c1ffc90c12e53dd7f4fde42a066ccea3...,2022-09-27 23:17:23,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,118788.939442
...,...,...,...,...,...,...,...,...,...,...,...
2023-06-07 13:42:47,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,0x0000000000000000000000000000000000000000,-2000.000000,Transfer,17428863,87,247,0xaf7747639152a7093b205d38308052f6fba17716c65c...,2023-06-07 13:42:47,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,107303.607420
2023-06-07 13:43:35,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,0x0000000000000000000000000000000000000000,-2000.000000,Transfer,17428867,163,204,0x7183029355114ef096e371caae8dc304e9c21f78cecc...,2023-06-07 13:43:35,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,105303.607420
2023-06-07 13:43:59,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,0x0000000000000000000000000000000000000000,-2000.000000,Transfer,17428869,70,133,0xfea33c7d1cf4a21e4269a595047bd551dbd8af65b5da...,2023-06-07 13:43:59,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,103303.607420


In [31]:
px.scatter(largest_whale['cumsol_col'])

In [32]:
# find dates of reward rate that are associated with the transaction dates of the largest whale 
reward_time_largest_whale = stETH_ETH_incentive_rewards_df.loc[stETH_ETH_incentive_rewards_df.index.isin(largest_whale['timestamp'].dt.normalize())]
reward_time_largest_whale

,lp_token_total_staked,reward_rate,block,reward_rate_eth
timestamp,,,,
2022-09-26,1.026352e+06,0.964506,15613828,0.001177
2022-09-27,1.172677e+06,0.964506,15620976,0.001160
2022-09-28,1.175841e+06,0.964506,15628117,0.001168
2022-09-29,1.176014e+06,0.964506,15635279,0.001150
2022-09-30,1.180506e+06,0.964506,15642433,0.001146
...,...,...,...,...
2023-05-30,6.009737e+05,0.106096,17368061,0.000121
2023-06-01,5.598626e+05,0.106096,17382266,0.000119
2023-06-02,5.585102e+05,0.106096,17389365,0.000128


In [33]:
reward_time_largest_whale = reward_time_largest_whale.sort_index()
reward_time_largest_whale

,lp_token_total_staked,reward_rate,block,reward_rate_eth
timestamp,,,,
2022-09-26,1.026352e+06,0.964506,15613828,0.001177
2022-09-27,1.172677e+06,0.964506,15620976,0.001160
2022-09-28,1.175841e+06,0.964506,15628117,0.001168
2022-09-29,1.176014e+06,0.964506,15635279,0.001150
2022-09-30,1.180506e+06,0.964506,15642433,0.001146
...,...,...,...,...
2023-05-30,6.009737e+05,0.106096,17368061,0.000121
2023-06-01,5.598626e+05,0.106096,17382266,0.000119
2023-06-02,5.585102e+05,0.106096,17389365,0.000128


In [34]:
# append onto largest whale df the reward rate column from reward time largest whale 
merged_largest_whale = pd.merge_asof(largest_whale, reward_time_largest_whale['reward_rate_eth'], left_index=True, right_index=True)
merged_largest_whale

,from,to,value,event,block,transaction_index,log_index,hash,timestamp,user,cumsol_col,reward_rate_eth
timestamp,,,,,,,,,,,,
2022-09-26 14:36:23,0x0000000000000000000000000000000000000000,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,118782.299389,Transfer,15618180,367,385,0x200853bc39a0b4ac57f3a396d22a39a070686ec5e63a...,2022-09-26 14:36:23,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,118782.299389,0.001177
2022-09-26 19:08:35,0x0000000000000000000000000000000000000000,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,0.921205,Transfer,15619528,39,152,0xe62463f53c97cc3542eb72887719022b33172209954b...,2022-09-26 19:08:35,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,118783.220593,0.001177
2022-09-27 04:24:35,0x0000000000000000000000000000000000000000,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,1.909552,Transfer,15622291,162,246,0xfb072e072c6557451effe6a61158eed4360decd747c2...,2022-09-27 04:24:35,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,118785.130146,0.001160
2022-09-27 13:44:23,0x0000000000000000000000000000000000000000,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,1.903760,Transfer,15625068,172,316,0x21ba8a9e96c6d327fb6df54ff59a683ed8dc418fa2c7...,2022-09-27 13:44:23,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,118787.033906,0.001160
2022-09-27 23:17:23,0x0000000000000000000000000000000000000000,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,1.905536,Transfer,15627904,76,160,0xa1add66f1689c1ffc90c12e53dd7f4fde42a066ccea3...,2022-09-27 23:17:23,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,118788.939442,0.001160
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-07 13:42:47,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,0x0000000000000000000000000000000000000000,-2000.000000,Transfer,17428863,87,247,0xaf7747639152a7093b205d38308052f6fba17716c65c...,2023-06-07 13:42:47,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,107303.607420,0.000132
2023-06-07 13:43:35,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,0x0000000000000000000000000000000000000000,-2000.000000,Transfer,17428867,163,204,0x7183029355114ef096e371caae8dc304e9c21f78cecc...,2023-06-07 13:43:35,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,105303.607420,0.000132
2023-06-07 13:43:59,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,0x0000000000000000000000000000000000000000,-2000.000000,Transfer,17428869,70,133,0xfea33c7d1cf4a21e4269a595047bd551dbd8af65b5da...,2023-06-07 13:43:59,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,103303.607420,0.000132


In [35]:
# graph largest whale cumsol and associated reward rates 
px.line(merged_largest_whale[['cumsol_col', 'reward_rate_eth']])


This whale's behavior seems to be relatively inelastic because even as the reward rate decreases over time, the amount of liquidity they have in the pool doesn't change

In [36]:
# locating id of unique largets whale 
largest_whale = transfer_df[transfer_df['user'] == '0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3']

# merging stETH ETH df onto largest whale df so we can access the reward rate and cumsol col in the same dataframe
reward_time_largest_whale = stETH_ETH_incentive_rewards_df.loc[stETH_ETH_incentive_rewards_df.index.isin(largest_whale['timestamp'].dt.normalize())]
reward_time_largest_whale = reward_time_largest_whale.sort_index()
merged_largest_whale = pd.merge_asof(largest_whale, reward_time_largest_whale['reward_rate_eth'], left_index=True, right_index=True)

# the actual graph of whale balance over time vs. reward rate 
import plotly.graph_objects as go 
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=merged_largest_whale.index, y=merged_largest_whale['cumsol_col'], name='cumsol_col'),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=merged_largest_whale.index, y=merged_largest_whale['reward_rate_eth'], name='reward_rate_eth'),
    secondary_y=True, 
)

fig.update_xaxes(title_text='timestamp')
fig.update_yaxes(title_text='balance over time', secondary_y=False)
fig.update_yaxes(title_text='reward rate (eth)', secondary_y=True)
fig.show()

In [37]:
top_10_whales  = transfer_df.groupby('user').max().sort_values('cumsol_col').tail(10)
top_10_whales

,from,to,value,event,block,transaction_index,log_index,hash,timestamp,cumsol_col
user,,,,,,,,,,
0x47cC445c8845F7186A1eaB87Ae5D60CDA69b630c,0x0000000000000000000000000000000000000000,0x47cC445c8845F7186A1eaB87Ae5D60CDA69b630c,80752.503711,Transfer,13229546,326,495,0xb0e09e87aba8325f1557630b3f9eb45c627ae043058e...,2021-09-15 09:37:53,80752.602372
0x6Cf9AA65EBaD7028536E353393630e2340ca6049,0x6Cf9AA65EBaD7028536E353393630e2340ca6049,0x6Cf9AA65EBaD7028536E353393630e2340ca6049,25064.782828,Transfer,15951773,510,440,0xfb8d3297f3e44c891d0e8acc1e794086656b77a9afa6...,2022-11-12 05:13:59,87377.445783
0x3BA21b6477F48273f41d241AA3722FFb9E07E247,0x3BA21b6477F48273f41d241AA3722FFb9E07E247,0x3BA21b6477F48273f41d241AA3722FFb9E07E247,50957.975751,Transfer,14759722,233,578,0xfd5dfd129c84021a54ad70d82c0d61ef6aa52b49129d...,2022-05-12 07:05:00,94362.289898
0x9026A229b535ecF0162Dfe48fDeb3c75f7b2A7AE,0x9026A229b535ecF0162Dfe48fDeb3c75f7b2A7AE,0x9026A229b535ecF0162Dfe48fDeb3c75f7b2A7AE,88054.537600,Transfer,16802027,68,204,0xfb84ba2eb60d3cb68ab2bbe9e5d54b8f1faee4ff6126...,2023-03-11 02:58:35,95615.757600
0x25AD8b7cd0738331E65b92736b71A4979691E479,0x0000000000000000000000000000000000000000,0x25AD8b7cd0738331E65b92736b71A4979691E479,98740.113301,Transfer,12484412,182,241,0x74639f49fd5fffd68cf9f156753a6480ea3dba606585...,2021-05-22 13:38:54,98740.113301
0x12b5c9191E186658841F2431943c47278f68E075,0x12b5c9191E186658841F2431943c47278f68E075,0x12b5c9191E186658841F2431943c47278f68E075,86061.625846,Transfer,14937471,273,587,0xef05f74c37b9b32cbfdaf80933c076b344a00af86bb0...,2022-06-10 08:37:37,105327.329795
0x9Cf59Ecde926cc1B5FF1Cb9b247E0FEC627091b1,0x9Cf59Ecde926cc1B5FF1Cb9b247E0FEC627091b1,0x9Cf59Ecde926cc1B5FF1Cb9b247E0FEC627091b1,60862.175792,Transfer,13188689,269,356,0xff8570fa3c8b4b43055ebc7bd984ab412e68a8295530...,2021-09-09 01:44:23,114142.222772
0xEAA7723633cf598E872D611f5EC50a45b65CBc72,0xEAA7723633cf598E872D611f5EC50a45b65CBc72,0xEAA7723633cf598E872D611f5EC50a45b65CBc72,114348.306873,Transfer,17512484,382,1100,0xf2c693113c6e9693eced243131d5e7dfe43c3148cf74...,2023-06-19 08:04:23,118845.280202
0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,119117.650376,Transfer,17428878,367,669,0xfea33c7d1cf4a21e4269a595047bd551dbd8af65b5da...,2023-06-07 13:45:47,119303.607420


Exclude zappers 

In [44]:
zapper_address = '0x271fbE8aB7f1fB262f81C77Ea5303F03DA9d3d6A'
zappers = transfer_df[(transfer_df['to'] == zapper_address) | (transfer_df['from'] == zapper_address)]
zappers['user'] = zappers['to']


In [46]:
# filter out rows matching zapper address 
transfer_df_filtered = transfer_df[~transfer_df.isin(zappers.to_dict('list')).all(1)]

# creating cumsol col in filtered transfer df 
transfer_df_filtered.sort_index(inplace=True)
transfer_df_filtered['cumsol_col'] = transfer_df_filtered.groupby(['user'])['value'].cumsum()


In [47]:
# the actual graph of whale balance over time vs. reward rate 
import plotly.graph_objects as go 
from plotly.subplots import make_subplots

# first get balances after 2022
transactions_after_2022 = transfer_df_filtered[transfer_df_filtered['timestamp'] > '2022-01-01']
# then select the top 21 whales
top_21_whales  = transactions_after_2022.groupby('user')[['cumsol_col']].max().sort_values('cumsol_col').tail(21)

for whale in top_21_whales.index:
    print(whale)
    largest_whale = transfer_df_filtered[transfer_df_filtered['user'] == whale]
    largest_whale['date'] = pd.to_datetime(largest_whale['timestamp'].dt.date)

    # to avoid duplicate days, we want the balance over time of 
    largest_whale = largest_whale.groupby('date')[['cumsol_col']].last()
    # resample here reindexes by day and then forward fills the values 
    largest_whale = largest_whale[['cumsol_col']].resample('D').ffill()
    
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(
    go.Bar(x=largest_whale.index, y=largest_whale['cumsol_col'].ffill(), name='balance over time'),
    secondary_y=False,
    )
    fig.add_trace(
    go.Scatter(x=stETH_ETH_incentive_rewards_df.index, y=stETH_ETH_incentive_rewards_df['reward_rate'], name='reward rate (LDO)'),
    secondary_y=True, 
    )

    fig.update_xaxes(title_text='timestamp')
    fig.update_yaxes(title_text='balance over time', secondary_y=False)
    fig.update_yaxes(title_text='reward rate (LDO)', secondary_y=True)
    fig.show()

0xC776207c8cD2A9d585E788AA28096AB803293F55


0x0c2f054331bDBEF366f390A343b3015f008488B3


0xa0c7a5e83552426dAE452469Ca6AdC138640f19e


0xb7d0EaBa9dCB38B3b94D7fa27164d5a5a8ED5812


0x5Ce9b49B7A1bE9f2c3DC2B2A5BaCEA56fa21FBeE


...

0x3BA21b6477F48273f41d241AA3722FFb9E07E247


0x9026A229b535ecF0162Dfe48fDeb3c75f7b2A7AE


0x12b5c9191E186658841F2431943c47278f68E075


0xEAA7723633cf598E872D611f5EC50a45b65CBc72


0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3


#### random whale and pivot information 

In [40]:
whale_df

,from,to,value,event,block,transaction_index,log_index,hash,timestamp,user,cumsol_col
timestamp,,,,,,,,,,,
2021-04-10 17:44:19,0x0000000000000000000000000000000000000000,0xdb5AA12AD695Ef2a28C6CdB69f2BB04BEd20a48e,31160.119571,Transfer,12213597,65,82,0xbc2b8f4aa968697352f7ab82f6db0dc30f0eb3b0559d...,2021-04-10 17:44:19,0xdb5AA12AD695Ef2a28C6CdB69f2BB04BEd20a48e,56051.962624
2021-04-12 19:07:06,0x0000000000000000000000000000000000000000,0xdb5AA12AD695Ef2a28C6CdB69f2BB04BEd20a48e,34577.303605,Transfer,12227029,0,0,0x277a7d200414999879bd2934b06b7cd144b62d181793...,2021-04-12 19:07:06,0xdb5AA12AD695Ef2a28C6CdB69f2BB04BEd20a48e,90629.266229
2021-04-13 17:33:41,0x0000000000000000000000000000000000000000,0xdb5AA12AD695Ef2a28C6CdB69f2BB04BEd20a48e,12772.157944,Transfer,12233123,161,183,0x0236cf92c4cca2673faea6a29d538c75cbbc9447ca05...,2021-04-13 17:33:41,0xdb5AA12AD695Ef2a28C6CdB69f2BB04BEd20a48e,103401.424173
2021-04-21 22:35:34,0x0000000000000000000000000000000000000000,0xdb5AA12AD695Ef2a28C6CdB69f2BB04BEd20a48e,13208.519255,Transfer,12286307,6,9,0xf1473c518b2a16f07411ab22a760bc3e88d0bbde1f50...,2021-04-21 22:35:34,0xdb5AA12AD695Ef2a28C6CdB69f2BB04BEd20a48e,116609.943428
2021-05-07 13:08:27,0x0000000000000000000000000000000000000000,0xdb5AA12AD695Ef2a28C6CdB69f2BB04BEd20a48e,29569.346367,Transfer,12387319,0,0,0xa7f2545100be96c67e71124ce06eda1d9eb5e2aac3c4...,2021-05-07 13:08:27,0xdb5AA12AD695Ef2a28C6CdB69f2BB04BEd20a48e,146179.289795
...,...,...,...,...,...,...,...,...,...,...,...
2023-06-16 13:05:35,0x0000000000000000000000000000000000000000,0x271fbE8aB7f1fB262f81C77Ea5303F03DA9d3d6A,46.525736,Transfer,17492581,124,166,0x248bb5fe3adc7f6ca6c60541fcd88073c25dcee2dd28...,2023-06-16 13:05:35,0x271fbE8aB7f1fB262f81C77Ea5303F03DA9d3d6A,57576.061964
2023-06-17 21:10:47,0x0000000000000000000000000000000000000000,0x271fbE8aB7f1fB262f81C77Ea5303F03DA9d3d6A,3.721728,Transfer,17502120,63,152,0xe05cde3d759ade71ff1948c18eeea9398148a3da3a39...,2023-06-17 21:10:47,0x271fbE8aB7f1fB262f81C77Ea5303F03DA9d3d6A,57579.783692
2023-06-17 21:18:11,0x0000000000000000000000000000000000000000,0x271fbE8aB7f1fB262f81C77Ea5303F03DA9d3d6A,18.608794,Transfer,17502157,87,226,0x015d8d951dfd2a2928e80c62b3ce62afaed808f3bd98...,2023-06-17 21:18:11,0x271fbE8aB7f1fB262f81C77Ea5303F03DA9d3d6A,57598.392485


In [41]:
px.line(whale_df['cumsol_col'])

In [42]:
pivot_df2 = transfer_df.pivot_table(values='cumsol_col', index='user')
pivot_df2 


,cumsol_col
user,
0x0000000000117D8C0c6103ecA077462224723192,-0.308512
0x000000000037D42ab4e2226CE6f44C5dC0Cf5b16,-37.188278
0x00000000245A821a3e646b8187636AD09Cd9cCB0,-85.109258
0x000000003fBC5761A6849dC46d6C292c7B01e894,-93.203660
0x0000000090E2bA937F0b5615b57409a8A6a1D6Dd,0.963734
...,...
0xff879CA9C84D976dB7E0180436e3a689eD70E1D2,0.000096
0xff8a51f0dE0166672a2f77f0213e6D814c28E36C,-36.296353
0xff9A717FCF03318485579c4b553820eDf73aDf0A,25.244343


In [43]:
pivot_df = pivot_df.ffill()

NameError: name 'pivot_df' is not defined

### when did this person enter and leave

In [ ]:
example_person = "0x80ee7c0e1e59929823eD9B22e34538b226967109"
add_liquidity_transfers[add_liquidity_transfers["to"] == example_person]

,from,to,value,event,block,transaction_index,log_index,hash,timestamp,user
timestamp,,,,,,,,,,
2021-01-12 22:16:32,0x0000000000000000000000000000000000000000,0x80ee7c0e1e59929823eD9B22e34538b226967109,1.999396,Transfer,11642841,30,51,0xb03636820959b9641a5c4c39676967180d7629a9bc30...,2021-01-12 22:16:32,0x80ee7c0e1e59929823eD9B22e34538b226967109
2021-01-12 22:31:11,0x0000000000000000000000000000000000000000,0x80ee7c0e1e59929823eD9B22e34538b226967109,0.999698,Transfer,11642902,120,187,0xcf8723e0451638d3f68a12395facc8c0f1ed2178a90f...,2021-01-12 22:31:11,0x80ee7c0e1e59929823eD9B22e34538b226967109


In [ ]:
withdraw_liquidity_transfers[withdraw_liquidity_transfers["from"] == example_person]

,from,to,value,event,block,transaction_index,log_index,hash,timestamp,user
timestamp,,,,,,,,,,
2021-01-14 13:13:07,0x80ee7c0e1e59929823eD9B22e34538b226967109,0x0000000000000000000000000000000000000000,-2.999095,Transfer,11653414,210,243,0x642a0a0273035551b6fa585c8615e129bc05772c9277...,2021-01-14 13:13:07,0x80ee7c0e1e59929823eD9B22e34538b226967109


This person entered twice at 2021-01-12 and then left at 2021-01-14. So they were only in for 3 days.

### Open Problems

Here are some problems you could look at. Remember that the focus is figuring out how changing incentive APR affacts LPs entering and leaving the pool

1. How does the amount of ETH value added influence the duration of a person's stay in the pool? (does this relationship change over time?)
2. Among those who fully exited the pool, what was the difference between the starting and ending Annual Percentage Rate (APR)?
3. What was the average duration of people's participation in the pool?
4. How frequently did people perform actions such as adding liquidity or withdrawing liquidity? Did they break up these actions into smaller increments?
5. Can the participants be clustered into meaningful groups based on their LP sizes and durations?
6. Are there any notable thresholds or breakpoints at which people exhibit different behaviors? eg. after a person has been in a pool for > 1 month they almost never leave
7. It is important to pay special attention to participants with large LPs. How do the people who have added and withdrawn large amounts of liquidity differ?

# Elasticity Subgroups

##### LP's who provided over 100 ETH 

In [ ]:
# merge add_lp with stETH_ETH_apr_df so that we can access the apr's 
add_lp['date'] = pd.to_datetime(add_lp['timestamp'].dt.date)
merged_df = pd.merge(add_lp, stETH_ETH_apr_df, left_on='date', right_on='timestamp', how='left') 
merged_df.set_index('date', inplace=True)

In [ ]:
# sum up tokens provided for each coin and select lp's that provided over 100 
merged_df['total_tokens'] = merged_df['token_amounts_0'] + merged_df['token_amounts_1']  
add_over_100_subgroup = merged_df.loc[merged_df['total_tokens'] > 100] 

subgroup_shift = add_over_100_subgroup.shift(-30, freq='D')
add_over_100_subgroup = pd.merge_asof(add_over_100_subgroup, subgroup_shift[['rolling_cur_naive_apr', 'token_supply']], left_index=True, right_index=True, direction='forward', suffixes=('', '_prev'))

In [ ]:
# set variables for elasticity formula
q2 = add_over_100_subgroup['token_supply']
q1 = add_over_100_subgroup['token_supply_prev']
p2 = add_over_100_subgroup['rolling_cur_naive_apr']
p1 = add_over_100_subgroup['rolling_cur_naive_apr_prev']

# calculate elasticity 
percent_change_supply = (q2 - q1) / ((q2 + q1) / 2) * 100 
percent_change_price = (p2 - p1) / ((p2 + p1) / 2) * 100 
add_over_100_subgroup['elasticity'] = percent_change_supply / percent_change_price 
add_over_100_subgroup['elasticity']

warnings.filterwarnings('ignore')

##### Mostly similar, but we're missing the large negative shock in May 2023 AND the positive shock is much larger 
- As apr increases, we expect quantity to increase, i.e. all elasticity values should be positive. This market seems to behave more rationally than the other market. 

In [ ]:
px.line(add_over_100_subgroup['elasticity'].dropna())

In [ ]:
px.line(add_over_100_subgroup['market_eth_tvl'].dropna())

NameError: name 'add_over_100_subgroup' is not defined

- In regards to the large positive shock, there could be outside factors unrelated to apr that cause ppl to leave/enter. For example, crazy low apr in other markets would lead to lots of people entering stETH ETH market because they're attracted to the higher return. 
- other markets unsafe

#### LP's that added liquidity when APR is greater than median percentage

In [ ]:
# convert to datetime so we don't lose data, then merge add_liquidity_transfers with stETH_ETH_apr_df SO WE CAN ACCESS APR'S 
add_liquidity_transfers['date_only'] = pd.to_datetime(add_liquidity_transfers.index.date)
merge2_df = pd.merge(add_liquidity_transfers, stETH_ETH_apr_df, left_on='date_only', right_on='timestamp', how='left')
merge2_df.set_index('date_only', inplace=True)


In [ ]:
# find where lp's added liquidity when apr is greater than median 
added_when_apr_over_med = merge2_df.loc[merge2_df['rolling_cur_naive_apr'] > merge2_df['rolling_cur_naive_apr'].median()]
# added_when_apr_over_med = added_when_apr_over_med[~added_when_apr_over_med.index.duplicated(keep='first')]

shift = added_when_apr_over_med.shift(-30, freq='D')
added_when_apr_over_med = pd.merge_asof(added_when_apr_over_med, shift[['rolling_cur_naive_apr', 'value']], left_index=True, right_index=True, direction='forward', suffixes=('', '_prev'))


In [ ]:
# set price variables 
p2 = added_when_apr_over_med['rolling_cur_naive_apr']
p1 = added_when_apr_over_med['rolling_cur_naive_apr_prev']

# set supply variables
q2 = added_when_apr_over_med['value']
q1 = added_when_apr_over_med['value_prev']

# calculate elasticity
percent_change_supply = (q2 - q1) / ((q2 + q1) / 2) * 100 
percent_change_price = (p2 - p1) / ((p2 + p1) / 2) * 100 
added_when_apr_over_med['elasticity'] = percent_change_supply / percent_change_price


In [ ]:
px.line(added_when_apr_over_med['elasticity'].dropna())

This elasticity does not seem to even be influenced by apr. The infinitely large elasticity values could indicate perfect elasticity or error in the code 

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=added_when_apr_over_med.index, y=added_when_apr_over_med['elasticity'], name='elasticity'), 
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=added_when_apr_over_med.index, y=added_when_apr_over_med['rolling_cur_naive_apr'], name='price (apr)'), 
    secondary_y=True,
)

# Set x-axis title
fig.update_xaxes(title_text="time stamp")

# Set y-axes titles
fig.update_yaxes(title_text="<b>elasticity</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>price (measured in apr)</b>", secondary_y=True)